# 1. Instalar librerias

In [2]:
#Crea el EDA
!pip install ydata-profiling
!pip install ipywidgets
#Arregla los caracteres especiales
!pip install ftfy

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.1/400.1 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 687.8/687.8 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 3.2 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27081 sha256=dea546dd313f722104ecbf3c160cb14daf79f20b560cf70a26a39ddea5e51de3
  Stored in directory: /root/.cache/pip/wheels/8d/55/1a/19cd535375ed1ede0c996405ebffe34b196d78e2d9545723a2
Successfully built htmlmin
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.6 MB/s eta 0:00:00


# 2. Cargar librerias

In [3]:
import pandas as pd
import chardet
from ydata_profiling import ProfileReport
import csv
import re
from datetime import datetime
from ftfy import fix_text
import math
import numpy as np
import os

# 3. Funciones

In [4]:
def DetectaEncoding(url):
  #Detectar el encoding de csv
  with open(url, 'rb') as f:
    result = chardet.detect(f.read(50000))
    encoding = result['encoding']
    if encoding.lower() == 'ascii':
      #Forzamos un encoding más permisivo
      encoding = 'latin-1'
    if encoding.lower() == 'big5':
      #Forzamos un encoding más permisivo
      encoding = 'cp1252'
  return encoding


def EDA(df, title):
  EDA = ProfileReport(df, title="EDA - " + title, explorative=True)
  EDA.to_notebook_iframe()
  #EDA_medicacion.to_file("EDA_Terrazas.html")
  #df1.to_csv('Terrazas_202104_EDA.csv', sep=";", encoding='latin-1', index=False)


def DetectaSeparador(url, num_lineas=5):
    with open(url, 'r', encoding=DetectaEncoding(url), errors='ignore') as f:
        muestra = ''.join([f.readline() for _ in range(num_lineas)])
        dialecto = csv.Sniffer().sniff(muestra)
        return dialecto.delimiter


# Función para limpiar y normalizar los formatos
def limpiar_fecha(fecha_str):
    #Nulos
    if pd.isna(fecha_str):
        return None

    #Limpiamos los espacios
    fecha_str = str(fecha_str).strip()

    # Reemplazar '-' por '/' para unificar formatos
    fecha_str = fecha_str.replace("/", "-")

    # Detectar formato "DD-MM-YYYY HH:MM"
    if re.fullmatch(r"\d{2}-\d{2}-\d{4} \d{2}:\d{2}", fecha_str):
        return f"{fecha_str}:00"

    # Detectar formato "DD-MM-YYYY H:MM"
    if re.fullmatch(r"\d{2}-\d{2}-\d{4} \d{1}:\d{2}", fecha_str):
        fecha, hora = fecha_str.split(" ")
        hora, minuto = hora.split(":")
        return f"{fecha} 0{hora}:{minuto}:00"

    # Detectar formato "YYYY-MM-DD HH:MM:SS.fffffffff",  invertirt año
    if re.fullmatch(r"\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}.\d{9}", fecha_str):
        fechahora, microsegundo = fecha_str.split(".")
        fecha, hora = fechahora.split(" ")
        y, m, d = fecha.split("-")
        return f"{d}-{m}-{y} {hora}"

    # Detectar formato "YYYY-MM-DD HH:MM",  invertirt año
    if re.fullmatch(r"\d{4}-\d{2}-\d{2} \d{2}:\d{2}", fecha_str):
        fecha, hora = fecha_str.split(" ")
        y, m, d = fecha.split("-")
        return f"{d}-{m}-{y} {hora}:00"

    # Detectar formato "YYYY-MM-DD HH:MM:SS",  invertirt año
    if re.fullmatch(r"\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}", fecha_str):
        fecha, hora = fecha_str.split(" ")
        y, m, d = fecha.split("-")
        return f"{d}-{m}-{y} {hora}"

    #No retornamos porque falta la hora,  invertirt año
    if re.fullmatch(r"\d{4}-\d{2}-\d{2}", fecha_str):
        y, m, d = fecha_str.split("-")
        fecha_str = f"{d}-{m}-{y} 00:00:00"

    # Si la fecha tiene formato tipo "10/10/2025" (sin hora)
    if re.fullmatch(r"\d{2}-\d{2}-\d{4}", fecha_str):
        fecha_str += " 00:00:00"

    return fecha_str

#limpiamos las horas
def limpiar_hora(hora_str):
    #Nulos
    if pd.isna(hora_str):
        return None
    # Convertir a string y limpiar espacios en blanco
    hora_str = str(hora_str).strip()

    if re.fullmatch(r"\d{1}:\d{2}", hora_str):
        hora, minuto = hora_str.split(":")
        return f"0{hora}:{minuto}:00"

    if re.fullmatch(r"\d{2}:\d{2}", hora_str):
        hora, minuto = hora_str.split(":")
        return f"{hora}:{minuto}:00"

    if re.fullmatch(r"\d{1}:\d{2}:\d{2}", hora_str):
        hora, minuto, segundos = hora_str.split(":")
        return f"0{hora}:{minuto}:{segundos}"

    return hora_str

#remplazamos caracteres especiales
def limpiar_texto(texto):
    if not pd.isna(texto):
        texto = fix_text(texto)
        texto = texto.replace('°', 'í')
        texto = texto.replace('‡', 'Ó')
        texto = texto.replace('ê', 'É')
        texto = texto.replace('ß', 'º')
        texto = texto.replace('÷', 'Í')
        texto = texto.replace('•', 'Ñ')
        texto = texto.replace('µ', 'Á')
        texto = texto.replace('Ã', 'Í')
        texto = texto.replace('Âº', '°')
        texto = texto.replace('¢', 'Ó')
        texto = texto.replace('Ç', 'É')
        texto = texto.replace('†', 'Á')
        texto = texto.replace('Ê', 'U')
        texto = texto.replace('\n', ' ')




        texto = texto.upper()
    return texto

#Convertimos columna en decimal con 1 decimal
def limpiar_decimal(decimal_str):
    if pd.isna(decimal_str):
       return None
    if decimal_str == '0' or decimal_str == '0.0':
        return None

    decimal_str = str(decimal_str).replace(',', '.')
    num = float(decimal_str)

    # Redondear hacia arriba a 1 decimal
    num_redondeado = math.ceil(num * 10) / 10
    return num_redondeado

#Convertimos una colomna en enteros nuleables
def limpiar_entero(columna):
    columna = columna.astype('Int64')
    columna = columna.replace(0, None)
    return columna


# 4. Directorios

In [5]:
#Direcorios

ruta = 'Limpios'

# verificamos si el directorio ya existe
if not os.path.exists(ruta):
  os.mkdir(ruta)
  print("Directorio %s creado!" % ruta)
else:
  print("Directorio %s ya existe" % ruta)

Directorio Limpios creado!


# 5. Limpiar

## 1. Ingresos

In [ ]:
url = "01_ingresos.csv";
df_ingresos_original =  pd.read_csv(url,delimiter=DetectaSeparador(url),encoding=DetectaEncoding(url))
df_ingresos_tratado = df_ingresos_original.copy()
df_ingresos_original

,ID_PACIENTE,EDAD,SEXO,DIAGNOSTICO_INGRESO,FECHA_INGRESO,FECHA_UCI_INGRESO,FECHA_UCI_SALIDA,UCI_DIAS,FECHA_ALTA_INGRESO,MOTIVO_ALTA_INGRESO,...,SAT_02_ULTIMA_URGENCIA,TA_MAX_ULTIMA_URGENCIA,TA_MIN_ULTIMA_URGENCIA,FUENTE,DIURESIS_PRIMERA_URGENCIA,IDCDSL,FECHA_INGRESO_ANTERIOR,DIAGNOSTICO_ANTERIOR,RESPIRADOR,UCI_N_ING
0,577,78,MALE,COVID19 - POSITIVO,26/12/2019,26/12/2019 17:12,27/12/2019 12:22,1.0,02/04/2020,Domicilio,...,0,0,0,F_24_04_2020,NaN,NaN,NaN,NaN,NaN,NaN
1,44,75,FEMALE,COVID19 - POSITIVO,28/01/2020,30/01/2020 13:03,31/01/2020 17:08,1.0,04/04/2020,Domicilio,...,0,0,0,F_24_04_2020,NaN,NaN,NaN,NaN,NaN,NaN
2,585,62,FEMALE,COVID19 - POSITIVO,05/02/2020,10/03/2020 14:20,20/03/2020 14:11,10.0,26/03/2020,Domicilio,...,97,160,101,F_24_04_2020,NaN,NaN,NaN,NaN,NaN,NaN
3,587,69,MALE,COVID19 - POSITIVO,06/02/2020,NaN,NaN,NaN,24/03/2020,Domicilio,...,95,141,83,F_24_04_2020,NaN,NaN,NaN,NaN,NaN,NaN
4,586,67,FEMALE,COVID19 - POSITIVO,06/02/2020,NaN,NaN,NaN,28/03/2020,Fallecimiento,...,93,151,82,F_24_04_2020,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9328,2742,61,MALE,COVID CONFIRMADO,2020-03-22 00:00:00,NaN,NaN,NaN,2020-03-24 00:00:00,Traslado al Hospital,...,88,130,88,F_19_04_2021,0.0,2742.0,2020-03-24,COVID CONFIRMADO,SI,NaN
9329,2743,54,MALE,COVID CONFIRMADO,2020-03-22 00:00:00,NaN,NaN,NaN,2020-04-02 00:00:00,Domicilio,...,87,108,73,F_19_04_2021,0.0,2743.0,NaN,NaN,SI,NaN
9330,2744,77,MALE,COVID CONFIRMADO,2020-03-23 00:00:00,NaN,NaN,NaN,2020-04-07 00:00:00,Domicilio,...,98,0,0,F_19_04_2021,0.0,2744.0,2020-04-10,NEUTROPENIA FEBRIL,SI,NaN
9331,2745,71,MALE,COVID CONFIRMADO,2020-03-22 00:00:00,2020-03-27 19:06:27.673000000,2020-04-27 13:32:57.683000000,31.0,2020-05-22 00:00:00,Domicilio,...,97,110,72,F_19_04_2021,0.0,2745.0,2020-07-09,AIT,SI,4.0


In [ ]:
#Transformar atributo EDAD en entero
df_ingresos_tratado["EDAD"] = df_ingresos_tratado["EDAD"].astype('Int64')
#Transformar atributo UCI_DIAS en entero
df_ingresos_tratado["UCI_DIAS"] = df_ingresos_tratado["UCI_DIAS"].astype('Int64')


In [ ]:
EDA(df_ingresos_tratado, "Ingresos")

In [ ]:
print(df_ingresos_tratado.value_counts("DIAGNOSTICO_INGRESO"))

#uNIFICAREMOS LOS DATOS EN 2 GRANDES GRUPOS

#COVID POSITIVO
#COVID SOSPECHA

df_ingresos_tratado["DIAGNOSTICO_INGRESO"] = df_ingresos_tratado["DIAGNOSTICO_INGRESO"].replace("COVID CONFIRMADO","COVID POSITIVO")
df_ingresos_tratado["DIAGNOSTICO_INGRESO"] = df_ingresos_tratado["DIAGNOSTICO_INGRESO"].replace("COVID19 - POSITIVO","COVID POSITIVO")
#df_ingresos_tratado["DIAGNOSTICO_INGRESO"] = df_ingresos_tratado["DIAGNOSTICO_INGRESO"].replace("COVID SOSPECHA","COVID SOSPECHA")
df_ingresos_tratado["DIAGNOSTICO_INGRESO"] = df_ingresos_tratado["DIAGNOSTICO_INGRESO"].replace("COVID19 - PENDIENTE","COVID SOSPECHA")

print("\n")
print(df_ingresos_tratado.value_counts("DIAGNOSTICO_INGRESO"))
print("\n")
print(df_ingresos_tratado['ID_PACIENTE'].count())

DIAGNOSTICO_INGRESO
COVID CONFIRMADO       6260
COVID19 - POSITIVO     2075
COVID SOSPECHA          766
COVID19 - PENDIENTE     232
Name: count, dtype: int64


DIAGNOSTICO_INGRESO
COVID POSITIVO    8335
COVID SOSPECHA     998
Name: count, dtype: int64


9333


In [ ]:
# Limpiar Fecha Ingreso
df_ingresos_tratado['FECHA_INGRESO'] = df_ingresos_tratado['FECHA_INGRESO'].apply(limpiar_fecha)
df_ingresos_tratado['FECHA_INGRESO'] = pd.to_datetime(df_ingresos_tratado['FECHA_INGRESO'], format="%d-%m-%Y %H:%M:%S")

In [ ]:
# Limpiar Fecha UCI ingreso
df_ingresos_tratado['FECHA_UCI_INGRESO'] = df_ingresos_tratado['FECHA_UCI_INGRESO'].apply(limpiar_fecha)
df_ingresos_tratado['FECHA_UCI_INGRESO'] = pd.to_datetime(df_ingresos_tratado['FECHA_UCI_INGRESO'], format="%d-%m-%Y %H:%M:%S")


In [ ]:
# Limpiar Fecha UCI salida
df_ingresos_tratado['FECHA_UCI_SALIDA'] = df_ingresos_tratado['FECHA_UCI_SALIDA'].apply(limpiar_fecha)
df_ingresos_tratado['FECHA_UCI_SALIDA'] = pd.to_datetime(df_ingresos_tratado['FECHA_UCI_SALIDA'], format="%d-%m-%Y %H:%M:%S")

In [ ]:
# Limpiar FECHA_ALTA_INGRESO
df_ingresos_tratado['FECHA_ALTA_INGRESO'] = df_ingresos_tratado['FECHA_ALTA_INGRESO'].apply(limpiar_fecha)
df_ingresos_tratado['FECHA_ALTA_INGRESO'] = pd.to_datetime(df_ingresos_tratado['FECHA_ALTA_INGRESO'], format="%d-%m-%Y %H:%M:%S")

In [ ]:
#FECHA_URGENCIA_INGRESO le añadimos la HORA_URGENCIA_INGRESO y eliminamos HORA_URGENCIA_INGRESO

df_ingresos_tratado['FECHA_URGENCIA_INGRESO'] = df_ingresos_tratado.FECHA_URGENCIA_INGRESO.str.cat(df_ingresos_tratado.HORA_URGENCIA_INGRESO, sep=" ")
df_ingresos_tratado = df_ingresos_tratado.drop(columns=['HORA_URGENCIA_INGRESO'])

df_ingresos_tratado['FECHA_URGENCIA_INGRESO'] = df_ingresos_tratado['FECHA_URGENCIA_INGRESO'].apply(limpiar_fecha)
df_ingresos_tratado['FECHA_URGENCIA_INGRESO'] = pd.to_datetime(df_ingresos_tratado['FECHA_URGENCIA_INGRESO'], format="%d-%m-%Y %H:%M:%S")


In [ ]:
#Remplazamos caracteres especiales ESPECIALIDAD_URGENCIA

df_ingresos_tratado['ESPECIALIDAD_URGENCIA'] = df_ingresos_tratado['ESPECIALIDAD_URGENCIA'].apply(limpiar_texto)
df_ingresos_tratado.value_counts("ESPECIALIDAD_URGENCIA")

,count
ESPECIALIDAD_URGENCIA,
MEDICINA DE URGENCIAS,8243
MEDICINA INTERNA,300
MEDICINA GENERAL,288
GINECOLOGÍA,40
PEDIATRÍA,37
MEDICINA INTENSIVA,25
TRAUMATOLOGÍA,22
CARDIOLOGÍA,8
CIRUGÍA GENERAL,2


In [ ]:
#Limpieza caracteres especiales DIAGNOSTICO_URGENCIA
df_ingresos_tratado['DIAGNOSTICO_URGENCIA'] = df_ingresos_tratado['DIAGNOSTICO_URGENCIA'].apply(limpiar_texto)

df_ingresos_tratado.value_counts("DIAGNOSTICO_URGENCIA").head(50)

,count
DIAGNOSTICO_URGENCIA,
DIFICULTAD RESPIRATORIA,4257
FIEBRE,1040
SOSPECHA COVID-19,960
CUADRO CATARRAL,791
TOS,544
MALESTAR GENERAL,346
MALESTAR,192
DETERIORO PACIENTE ONCOLÓGICO,166
DOLOR ABDOMINAL,65


In [ ]:
#mayuscula MOTIVO_ALTA_INGRESO
df_ingresos_tratado['MOTIVO_ALTA_INGRESO'] = df_ingresos_tratado['MOTIVO_ALTA_INGRESO'].str.upper()
df_ingresos_tratado.value_counts("MOTIVO_ALTA_INGRESO")

,count
MOTIVO_ALTA_INGRESO,
DOMICILIO,7075
FALLECIMIENTO,1264
TRASLADO AL HOSPITAL,336
TRASLADO A UN CENTRO SOCIOSANITARIO,160
ALTA VOLUNTARIA,30
FUGA,1


In [ ]:
#mayuscula DESTINO_URGENCIA
df_ingresos_tratado['DESTINO_URGENCIA'] = df_ingresos_tratado['DESTINO_URGENCIA'].str.upper()
df_ingresos_tratado.value_counts("DESTINO_URGENCIA")

,count
DESTINO_URGENCIA,
INGRESO,8946
DOMICILIO,10
TRASLADO A HOSPITAL DEL GRUPO,5
TRASLADO A OTROS HOSPITALES,4
ALTA VOLUNTARIA POR MOTIVOS PERSONALES,1
ALTA VOLUNTARIA,1
ALTA VOLUNTARIA POR FALTA DE CAMAS,1


In [ ]:
#Limpiamos HORA_CONSTANTES_PRIMERA_URGENCIA
df_ingresos_tratado['HORA_CONSTANTES_PRIMERA_URGENCIA'] = df_ingresos_tratado['HORA_CONSTANTES_PRIMERA_URGENCIA'].apply(limpiar_hora)
df_ingresos_tratado.value_counts("HORA_CONSTANTES_PRIMERA_URGENCIA").tail(10)

,count
HORA_CONSTANTES_PRIMERA_URGENCIA,
08:57:00,1
08:04:00,1
22:38:00,1
23:40:00,1
23:19:00,1
23:28:00,1
00:05:00,1
00:15:00,1
23:58:00,1


In [ ]:
#df_ingresos_tratado = df_ingresos_original.copy()
df_ingresos_tratado['TEMPERATURA_PRIMERA_URGENCIA'] = df_ingresos_tratado['TEMPERATURA_PRIMERA_URGENCIA'].apply(limpiar_decimal)
df_ingresos_tratado.value_counts("TEMPERATURA_PRIMERA_URGENCIA")


,count
TEMPERATURA_PRIMERA_URGENCIA,
36.0,576
36.5,565
36.6,513
36.3,469
36.4,447
36.2,381
36.7,374
36.8,360
36.9,315


In [ ]:
#df_ingresos_tratado['TEMPERATURA_PRIMERA_URGENCIA'] = df_ingresos_tratado['TEMPERATURA_PRIMERA_URGENCIA'].apply(limpiar_decimal)
df_ingresos_tratado["FC_PRIMERA_URGENCIA"] = limpiar_entero(df_ingresos_tratado["FC_PRIMERA_URGENCIA"])
#df_ingresos_tratado.value_counts("FC_PRIMERA_URGENCIA")
df_ingresos_tratado["GLU_PRIMERA_URGENCIA"] = limpiar_entero(df_ingresos_tratado["GLU_PRIMERA_URGENCIA"])
#df_ingresos_tratado.value_counts("GLU_PRIMERA_URGENCIA")
df_ingresos_tratado["SAT_02_PRIMERA_URGENCIA"] = limpiar_entero(df_ingresos_tratado["SAT_02_PRIMERA_URGENCIA"])
#df_ingresos_tratado.value_counts("SAT_02_PRIMERA_URGENCIA")
df_ingresos_tratado["TA_MAX_PRIMERA_URGENCIA"] = limpiar_entero(df_ingresos_tratado["TA_MAX_PRIMERA_URGENCIA"])
#df_ingresos_tratado.value_counts("TA_MAX_PRIMERA_URGENCIA")
df_ingresos_tratado["TA_MIN_PRIMERA_URGENCIA"] = limpiar_entero(df_ingresos_tratado["TA_MIN_PRIMERA_URGENCIA"])
#df_ingresos_tratado.value_counts("TA_MIN_PRIMERA_URGENCIA")

In [ ]:
df_ingresos_tratado['HORA_CONSTANTES_ULTIMA_URGENCIA'] = df_ingresos_tratado['HORA_CONSTANTES_ULTIMA_URGENCIA'].apply(limpiar_hora)
df_ingresos_tratado.value_counts("HORA_CONSTANTES_ULTIMA_URGENCIA").head(10)

,count
HORA_CONSTANTES_ULTIMA_URGENCIA,
19:24:00,38
11:26:00,29
22:36:00,28
19:45:00,27
20:35:00,26
19:15:00,26
18:38:00,26
21:00:00,26
18:59:00,25


In [ ]:
df_ingresos_tratado['TEMPERATURA_ULTIMA_URGENCIA'] = df_ingresos_tratado['TEMPERATURA_ULTIMA_URGENCIA'].apply(limpiar_decimal)
df_ingresos_tratado.value_counts("TEMPERATURA_ULTIMA_URGENCIA")

,count
TEMPERATURA_ULTIMA_URGENCIA,
36.5,1011
36.3,834
36.0,756
36.8,728
37.0,680
36.6,531
37.3,381
36.1,296
37.5,277


In [ ]:
#GLU_ULTIMA_URGENCIA
#SAT_02_ULTIMA_URGENCIA
#TA_MAX_ULTIMA_URGENCIA
#TA_MIN_ULTIMA_URGENCIA
df_ingresos_tratado['FC_ULTIMA_URGENCIA'] = limpiar_entero(df_ingresos_tratado["FC_ULTIMA_URGENCIA"])
# df_ingresos_tratado.value_counts("FC_ULTIMA_URGENCIA")
df_ingresos_tratado["GLU_ULTIMA_URGENCIA"] = limpiar_entero(df_ingresos_tratado["GLU_ULTIMA_URGENCIA"])
#df_ingresos_tratado.value_counts("GLU_ULTIMA_URGENCIA")
df_ingresos_tratado['SAT_02_ULTIMA_URGENCIA'] = limpiar_entero(df_ingresos_tratado['SAT_02_ULTIMA_URGENCIA'])
#df_ingresos_tratado.value_counts("SAT_02_ULTIMA_URGENCIA")
df_ingresos_tratado['TA_MAX_ULTIMA_URGENCIA'] = limpiar_entero(df_ingresos_tratado['TA_MAX_ULTIMA_URGENCIA'])
#df_ingresos_tratado.value_counts("TA_MAX_ULTIMA_URGENCIA")
df_ingresos_tratado['TA_MIN_ULTIMA_URGENCIA'] = limpiar_entero(df_ingresos_tratado['TA_MIN_ULTIMA_URGENCIA'])
#df_ingresos_tratado.value_counts("TA_MIN_ULTIMA_URGENCIA")

In [ ]:
#eliminamos IDCDSL
df_ingresos_tratado = df_ingresos_tratado.drop(columns=['IDCDSL'])

In [ ]:
# Limpiar FECHA_ALTA_INGRESO
df_ingresos_tratado['FECHA_INGRESO_ANTERIOR'] = df_ingresos_tratado['FECHA_INGRESO_ANTERIOR'].apply(limpiar_fecha)
df_ingresos_tratado['FECHA_INGRESO_ANTERIOR'] = pd.to_datetime(df_ingresos_tratado['FECHA_INGRESO_ANTERIOR'], format="%d-%m-%Y %H:%M:%S")

In [ ]:
#convertir en booleano
#df_ingresos_tratado = df_ingresos_original.copy()
df_ingresos_tratado['RESPIRADOR'] = df_ingresos_tratado['RESPIRADOR'].fillna('NO').str.strip().str.upper() == "SI"
df_ingresos_tratado['RESPIRADOR'].value_counts()


,count
RESPIRADOR,
False,5639
True,3694


In [ ]:
df_ingresos_tratado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9333 entries, 0 to 9332
Data columns (total 34 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   ID_PACIENTE                       9333 non-null   int64         
 1   EDAD                              9333 non-null   Int64         
 2   SEXO                              9333 non-null   object        
 3   DIAGNOSTICO_INGRESO               9333 non-null   object        
 4   FECHA_INGRESO                     9333 non-null   datetime64[ns]
 5   FECHA_UCI_INGRESO                 845 non-null    datetime64[ns]
 6   FECHA_UCI_SALIDA                  827 non-null    datetime64[ns]
 7   UCI_DIAS                          3146 non-null   Int64         
 8   FECHA_ALTA_INGRESO                9112 non-null   datetime64[ns]
 9   MOTIVO_ALTA_INGRESO               8866 non-null   object        
 10  FECHA_URGENCIA_INGRESO            8968 non-null 

In [ ]:
#exportamos en formato utf-8 reconocido por Excel
df_ingresos_tratado.to_csv(ruta + '/01_ingresos_limpio.csv', sep=";", encoding='utf-8-sig', index=False)

## 2. Medicación

In [ ]:
url = "02_medicacion.csv";
df_medicacion_original =  pd.read_csv(url,delimiter=DetectaSeparador(url),encoding=DetectaEncoding(url))
df_medicacion_tratado = df_medicacion_original.copy()
df_medicacion_original

,ID_PACIENTE,FARMACO_NOMBRE_COMERCIAL,DOSIS_MEDIA_DIARIA,INICIO_TRATAMIENTO,FIN_TRATAMIENTO,ATC5_NOMBRE,ID_ATC5,ATC7_NOMBRE,ID_ATC7,FUENTE
0,1,MORFINA amp 1% 1 mL (10 mg/1 mL),5,10/04/2020,10/04/2020,ALCALOIDES NATURALES DEL OPIO,N02AA,MORFINA,N02AA01,F_24_04_2020
1,1,DOLQUINE comp 200 mg,2,06/04/2020,10/04/2020,AMINOQUINOLINAS,P01BA,HIDROXICLOROQUINA,P01BA02,F_24_04_2020
2,1,PARACETAMOL comp 1 g,3,07/04/2020,10/04/2020,ANILIDAS,N02BE,PARACETAMOL,N02BE01,F_24_04_2020
3,1,KALETRA/ALUVIA comp (200+50) mg,4,07/04/2020,07/04/2020,ANTIVIRALES PARA EL TRATAMIENTO DE INFECCIONES...,J05AR,RITONAVIR Y LOPINAVIR,J05AE20,F_24_04_2020
4,1,DESINCLOR JABONOSO sol 4% F/500 mL,2,06/04/2020,06/04/2020,BIGUANIDAS Y AMIDINAS,D08AC,CLORHEXIDINA,D08AC02,F_24_04_2020
...,...,...,...,...,...,...,...,...,...,...
251736,2788,SUERO FISIOLOGICO LAV. 100 mL,1,2020-04-02,2020-04-02,SOLUCIONES ELECTROLÍTICAS,B05XA,CLORURO DE SODIO,B05XA03,F_19_04_2021
251737,2789,AZITROMICINA comp 500 mg,1,2020-03-24,2020-03-27,MACRÓLIDOS,J01FA,AZITROMICINA,J01FA10,F_19_04_2021
251738,2789,OXIGENO PLANTA,22,2020-03-23,2020-04-03,GASES MEDICINALES,V03AN,OXÍGENO,V03AN01,F_19_04_2021
251739,2789,SUERO ClNa 100 mL I.V.,2,2020-03-23,2020-04-03,SOLUCIONES QUE AFECTAN EL BALANCE ELECTROLÍTICO,B05BB,ELECTROLITOS,B05BB01,F_19_04_2021


In [ ]:
EDA(df_medicacion_tratado, "Medicación")

In [ ]:
df_medicacion_tratado['FARMACO_NOMBRE_COMERCIAL'] = df_medicacion_tratado['FARMACO_NOMBRE_COMERCIAL'].apply(limpiar_texto)
df_medicacion_tratado['FARMACO_NOMBRE_COMERCIAL'] = df_medicacion_tratado['FARMACO_NOMBRE_COMERCIAL'].str.strip().replace('gg.', 'gg')
df_medicacion_tratado['FARMACO_NOMBRE_COMERCIAL'] = df_medicacion_tratado['FARMACO_NOMBRE_COMERCIAL'].str.strip().replace('susp.', 'susp')
df_medicacion_tratado['FARMACO_NOMBRE_COMERCIAL'] = df_medicacion_tratado['FARMACO_NOMBRE_COMERCIAL'].str.strip().replace('sb.', 'sb')
df_medicacion_tratado['FARMACO_NOMBRE_COMERCIAL'] = df_medicacion_tratado['FARMACO_NOMBRE_COMERCIAL'].str.strip().replace('cap 0,', 'cap 0.')
df_medicacion_tratado['FARMACO_NOMBRE_COMERCIAL'] = df_medicacion_tratado['FARMACO_NOMBRE_COMERCIAL'].str.strip().replace('1L,', '1 L')
#remplazo numero,numero por numero.numero
df_medicacion_tratado['FARMACO_NOMBRE_COMERCIAL'] = df_medicacion_tratado['FARMACO_NOMBRE_COMERCIAL'].apply(
    lambda x: re.sub(r'(?<!\d)(\d+),(\d+)(?!\d)', r'\1.\2', str(x))
)


df_medicacion_tratado.value_counts("FARMACO_NOMBRE_COMERCIAL").head(50)

,count
FARMACO_NOMBRE_COMERCIAL,
SUERO CLNA 100 ML I.V.,8570
ALCOHOL 70º 1L,8342
PARACETAMOL SOL 1 G/100 ML,7949
OXIGENO PLANTA,7882
SUERO CLNA 500 ML I.V.,7182
SUERO CLNA 50 ML I.V.,7092
CLORURO SODICO AMP 0.9% 10 ML,7056
DESINCLOR JABONOSO SOL 4% F/500 ML,6549
SUERO GLUCOSADO 5% 500 ML,6203


In [ ]:
df_ingresos_tratado = df_ingresos_original.copy()
#los negativos los transformamos a positivos
df_medicacion_tratado['DOSIS_MEDIA_DIARIA'] = df_medicacion_tratado['DOSIS_MEDIA_DIARIA'].abs()
#df_medicacion_tratado.value_counts("DOSIS_MEDIA_DIARIA")

# df_medicacion_tratado[df_medicacion_tratado['ID_PACIENTE'] == 1645]

,ID_PACIENTE,FARMACO_NOMBRE_COMERCIAL,DOSIS_MEDIA_DIARIA,INICIO_TRATAMIENTO,FIN_TRATAMIENTO,ATC5_NOMBRE,ID_ATC5,ATC7_NOMBRE,ID_ATC7,FUENTE


In [ ]:
#INICIO_TRATAMIENTO
df_medicacion_tratado['INICIO_TRATAMIENTO'] = df_medicacion_tratado['INICIO_TRATAMIENTO'].apply(limpiar_fecha)
df_medicacion_tratado['INICIO_TRATAMIENTO'] = pd.to_datetime(df_medicacion_tratado['INICIO_TRATAMIENTO'], format="%d-%m-%Y %H:%M:%S")
df_medicacion_tratado.value_counts('INICIO_TRATAMIENTO')

,count
INICIO_TRATAMIENTO,
2020-03-24,8478
2020-03-30,7611
2020-03-25,7415
2020-03-27,7329
2020-03-26,7244
...,...
2017-12-16,1
2020-07-28,1
2020-06-20,1


In [ ]:
#FIN_TRATAMIENTO
df_medicacion_tratado['FIN_TRATAMIENTO'] = df_medicacion_tratado['FIN_TRATAMIENTO'].apply(limpiar_fecha)
df_medicacion_tratado['FIN_TRATAMIENTO'] = pd.to_datetime(df_medicacion_tratado['FIN_TRATAMIENTO'], format="%d-%m-%Y %H:%M:%S")
df_medicacion_tratado.value_counts('FIN_TRATAMIENTO')

,count
FIN_TRATAMIENTO,
2020-04-02,7280
2020-04-01,7214
2020-03-30,6823
2020-03-26,6736
2020-04-06,6554
...,...
2017-12-16,2
2020-07-11,2
2020-07-19,2


In [ ]:
#ATC5_NOMBRE
df_medicacion_tratado['ATC5_NOMBRE'] = df_medicacion_tratado['ATC5_NOMBRE'].apply(limpiar_texto)
df_medicacion_tratado.value_counts('ATC5_NOMBRE').tail(50)

,count
ATC5_NOMBRE,
ALCALOIDES DEL ERGOT,4
SIMPATICOMIMÉTICOS QUE INHIBEN EL TRABAJO DE PARTO,4
DERIVADOS DEL TIOXANTENO,3
DERIVADOS DE LA PIPERAZINA,3
DANTROLENO Y DERIVADOS,3
DERIVADOS DE LA FENOTIAZINA,3
DERIVADOS DE LA XANTINA,3
HORMONAS PARATIRÓIDEAS Y ANÁLOGAS,3
"BARBITÚRICOS, MONODROGAS",3


In [ ]:
#ATC7_NOMBRE
df_medicacion_tratado['ATC7_NOMBRE'] = df_medicacion_tratado['ATC7_NOMBRE'].apply(limpiar_texto)
df_medicacion_tratado.value_counts('ATC7_NOMBRE').tail(50)

,count
ATC7_NOMBRE,
FESOTERODINA,2
FENILEFRINA,2
IRBESARTÁN,2
MOXIFLOXACINA,2
BROMURO DE TIOTROPIO,2
MILRINONA,2
"MICONAZOL, COMBINACIONES",2
KETAZOLAM,2
MELATONINA,2


In [ ]:
df_medicacion_tratado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251741 entries, 0 to 251740
Data columns (total 10 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   ID_PACIENTE               251741 non-null  int64 
 1   FARMACO_NOMBRE_COMERCIAL  251741 non-null  object
 2   DOSIS_MEDIA_DIARIA        251741 non-null  int64 
 3   INICIO_TRATAMIENTO        251741 non-null  object
 4   FIN_TRATAMIENTO           251741 non-null  object
 5   ATC5_NOMBRE               251741 non-null  object
 6   ID_ATC5                   251741 non-null  object
 7   ATC7_NOMBRE               251736 non-null  object
 8   ID_ATC7                   251736 non-null  object
 9   FUENTE                    251741 non-null  object
dtypes: int64(2), object(8)
memory usage: 19.2+ MB


In [ ]:
df_medicacion_tratado.to_csv(ruta + '/02_medicacion_limpio.csv', sep=";", encoding='utf-8-sig', index=False)

## 3. Constantes Vitales

In [ ]:
url = "03_constantes.csv";
df_constantes_original =  pd.read_csv(url,delimiter=DetectaSeparador(url),encoding=DetectaEncoding(url))
df_constantes_tratado = df_constantes_original.copy()
df_constantes_original

,ID_PACIENTE,FECHA_CONSTANTE_INGRESO,HORA_CONSTANTE_INGRESO,FC_INRESO,GLU_INGRESO,SAT_02_INGRESO,TA_MAX_INGRESO,TA_MIN_INGRESO,TEMPERATURA_INGRESO,FUENTE,SAT_02_INGRESO_OBSERVACIONES
0,2316,24/04/2020,00:00,62,0,0,0,0,36.5,F_24_04_2020,NaN
1,2316,24/04/2020,00:01,69,0,0,0,0,37.1,F_24_04_2020,NaN
2,2288,24/04/2020,00:04,94,0,0,0,0,36.8,F_24_04_2020,NaN
3,514,24/04/2020,00:09,0,0,0,0,0,35.4,F_24_04_2020,NaN
4,181,24/04/2020,00:10,0,0,0,0,0,36.4,F_24_04_2020,NaN
...,...,...,...,...,...,...,...,...,...,...,...
407661,542,2020-05-17,10:27,0,0,98,0,0,NaN,F_19_04_2021,SATO2 GN 1L
407662,605,2020-05-17,11:55,0,0,96,0,0,NaN,F_19_04_2021,NaN
407663,580,2020-11-08,15:46,0,0,35,0,0,NaN,F_19_04_2021,SATO2 GN 4L
407664,4298,2020-11-08,15:50,0,0,97,0,0,NaN,F_19_04_2021,SATO2 GN 1L


In [ ]:
EDA(df_constantes_tratado, "Constantes Vitales")

In [ ]:
#INICIO_TRATAMIENTO
df_constantes_tratado['FECHA_CONSTANTE_INGRESO'] = df_constantes_tratado.FECHA_CONSTANTE_INGRESO.str.cat(df_constantes_tratado.HORA_CONSTANTE_INGRESO, sep=" ")
df_constantes_tratado = df_constantes_tratado.drop(columns=['HORA_CONSTANTE_INGRESO'])

df_constantes_tratado['FECHA_CONSTANTE_INGRESO'] = df_constantes_tratado['FECHA_CONSTANTE_INGRESO'].apply(limpiar_fecha)
df_constantes_tratado['FECHA_CONSTANTE_INGRESO'] = pd.to_datetime(df_constantes_tratado['FECHA_CONSTANTE_INGRESO'], format="%d-%m-%Y %H:%M:%S")


In [ ]:
#rename FC_INRESO to FC_INGRESO
df_constantes_tratado = df_constantes_tratado.rename(columns={'FC_INRESO': 'FC_INGRESO'})

In [ ]:
df_constantes_tratado["FC_INGRESO"] = limpiar_entero(df_constantes_tratado["FC_INGRESO"])
#df_constantes_tratado.value_counts("FC_INGRESO")
df_constantes_tratado["GLU_INGRESO"] = limpiar_entero(df_constantes_tratado["GLU_INGRESO"])
#df_constantes_tratado.value_counts("GLU_INGRESO")
df_constantes_tratado["SAT_02_INGRESO"] = limpiar_entero(df_constantes_tratado["SAT_02_INGRESO"])
#df_constantes_tratado.value_counts("SAT_02_INGRESO")
df_constantes_tratado["TA_MAX_INGRESO"] = limpiar_entero(df_constantes_tratado["TA_MAX_INGRESO"])
#df_constantes_tratado.value_counts("TA_MAX_INGRESO")
df_constantes_tratado["TA_MIN_INGRESO"] = limpiar_entero(df_constantes_tratado["TA_MIN_INGRESO"])
# df_constantes_tratado.value_counts("TA_MIN_INGRESO")
df_constantes_tratado['TEMPERATURA_INGRESO'] = df_constantes_tratado['TEMPERATURA_INGRESO'].apply(limpiar_decimal)
df_constantes_tratado.value_counts("TEMPERATURA_INGRESO")

,count
TEMPERATURA_INGRESO,
0.0,20361
36.0,17670
36.3,11790
36.1,11479
36.4,11440
...,...
33.0,2
32.2,2
31.1,2


In [ ]:
#SAT_02_INGRESO_OBSERVACIONES
df_constantes_tratado['SAT_02_INGRESO_OBSERVACIONES'] = df_constantes_tratado['SAT_02_INGRESO_OBSERVACIONES'].apply(limpiar_texto)
df_constantes_tratado.value_counts("SAT_02_INGRESO_OBSERVACIONES")

,count
SAT_02_INGRESO_OBSERVACIONES,
,180771
SATO2 BASAL,43730
SATO2 GN 2L,13248
SATO2 GN 4L,6883
SATO2 GN 3L,6144
...,...
SATO2 VMK 12L 45%,1
SATO2 VMK 26% 8LPM,1
SATO2 VMK 15 LITROS,1


In [ ]:
df_constantes_tratado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 407666 entries, 0 to 407665
Data columns (total 10 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   ID_PACIENTE                   407666 non-null  int64         
 1   FECHA_CONSTANTE_INGRESO       407666 non-null  datetime64[ns]
 2   FC_INGRESO                    171968 non-null  Int64         
 3   GLU_INGRESO                   0 non-null       Int64         
 4   SAT_02_INGRESO                171312 non-null  Int64         
 5   TA_MAX_INGRESO                109834 non-null  Int64         
 6   TA_MIN_INGRESO                109793 non-null  Int64         
 7   TEMPERATURA_INGRESO           186587 non-null  float64       
 8   FUENTE                        407666 non-null  object        
 9   SAT_02_INGRESO_OBSERVACIONES  329016 non-null  object        
dtypes: Int64(5), datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 33

In [ ]:
df_constantes_tratado.to_csv(ruta + '/03_constantes_limpio.csv', sep=";", encoding='utf-8-sig', index=False)

## 4. Laboratorio

In [75]:
url = "04_laboratorio.csv";
df_laboratorio_original =  pd.read_csv(url,delimiter=DetectaSeparador(url),encoding=DetectaEncoding(url))
df_laboratorio_tratado = df_laboratorio_original.copy()
df_laboratorio_original

,ID_PACIENTE,LAB_NUMERO,LAB_FECHA,LAB_HORA,LAB_ITEM,RESULTADO_VALOR,RESULTADO_UNIDAD,RESULTADO_VALORES_REFERENCIALES,FUENTE
0,1,I-1,08-04-2020,00:00,GOT -- GOT (AST),215.0,U/L,<41,F_24_04_2020
1,1,I-1,08-04-2020,00:00,LIN% -- Linfocitos %,3.0,%,(20.0 - 50.0),F_24_04_2020
2,1,I-1,08-04-2020,00:00,MONO -- Monocitos,0.12,x10e3/µL,(0.1 - 0.8),F_24_04_2020
3,1,I-2,07-04-2020,00:00,BAS -- Basófilos,0.02,x10e3/µL,(0.0 - 0.3),F_24_04_2020
4,1,I-2,07-04-2020,00:00,BT -- BILIRRUBINA TOTAL ...,0.43,mg/dL,<1.2,F_24_04_2020
...,...,...,...,...,...,...,...,...,...
1767077,4474,I-1,04-12-2020,00:00,G-CORONAV (RT-PCR) -- Tipo de muestra: EXUDADO,No se detecta,NaN,NaN,F_19_04_2021
1767078,4474,I-2,10-12-2020,00:00,G-CORONAV (RT-PCR) -- Tipo de muestra: EXUDADO,Se detecta,NaN,NaN,F_19_04_2021
1767079,4475,I-1,10-12-2020,00:00,G-CORONAV (RT-PCR) -- Tipo de muestra: EXUDADO,No se detecta,NaN,NaN,F_19_04_2021
1767080,4478,I-1,04-12-2020,00:00,G-CORONAV (RT-PCR) -- Tipo de muestra: EXUDADO,Se detecta,NaN,NaN,F_19_04_2021


In [ ]:
EDA(df_laboratorio_tratado, "Laboratorios")

In [51]:
df_laboratorio_tratado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1767082 entries, 0 to 1767081
Data columns (total 9 columns):
 #   Column                           Dtype 
---  ------                           ----- 
 0   ID_PACIENTE                      int64 
 1   LAB_NUMERO                       object
 2   LAB_FECHA                        object
 3   LAB_HORA                         object
 4   LAB_ITEM                         object
 5   RESULTADO_VALOR                  object
 6   RESULTADO_UNIDAD                 object
 7   RESULTADO_VALORES_REFERENCIALES  object
 8   FUENTE                           object
dtypes: int64(1), object(8)
memory usage: 121.3+ MB


In [76]:
#LAB_FECHA Y LAB_HORA
#df_laboratorio_tratado['LAB_FECHA2'] = df_laboratorio_tratado.LAB_FECHA.str.cat(df_laboratorio_tratado.LAB_HORA, sep=" ")
df_laboratorio_tratado['LAB_FECHA'] = df_laboratorio_tratado.LAB_FECHA.str.cat(df_laboratorio_tratado.LAB_HORA, sep=" ")
df_laboratorio_tratado = df_laboratorio_tratado.drop(columns=['LAB_HORA'])

df_laboratorio_tratado['LAB_FECHA'] = df_laboratorio_tratado['LAB_FECHA'].apply(limpiar_fecha)

#no lo transformo en datetiem, porque pandas al momento de exportar las fechas con horas con 00:00:00 se exportan sin horas
df_laboratorio_tratado['LAB_FECHA'] = pd.to_datetime(df_laboratorio_tratado['LAB_FECHA'], format="%d-%m-%Y %H:%M:%S")

In [60]:
df_laboratorio_tratado[(df_laboratorio_tratado['ID_PACIENTE'] == 621) & (df_laboratorio_tratado['LAB_NUMERO'] == 'I-96')]

,ID_PACIENTE,LAB_NUMERO,LAB_FECHA,LAB_ITEM,RESULTADO_VALOR,RESULTADO_UNIDAD,RESULTADO_VALORES_REFERENCIALES,FUENTE
1146928,621,I-96,2020-04-13,BE(b) -- BE(b),4.7,mmol/L,(-2.0 - 3.0),F_19_04_2021
1146929,621,I-96,2020-04-13,BE(b) -- GASOMETRIA ARTERIAL,4.7,mmol/L,(-2.0 - 3.0),F_19_04_2021
1146930,621,I-96,2020-04-13,BEecf -- BEecf,5.6,mmol/L,(-2.0 - 3.0),F_19_04_2021
1146931,621,I-96,2020-04-13,BEecf -- GASOMETRIA ARTERIAL,5.6,mmol/L,(-2.0 - 3.0),F_19_04_2021
1146932,621,I-96,2020-04-13,HCO3 -- GASOMETRIA ARTERIAL,30.4,mmol/L,(22.0 - 26.0),F_19_04_2021
1146933,621,I-96,2020-04-13,HCO3 -- HCO3-,30.4,mmol/L,(22.0 - 26.0),F_19_04_2021
1146934,621,I-96,2020-04-13,LAC -- LACTATO,0.9,mmol/L,(0.5 - 2.4),F_19_04_2021
1146935,621,I-96,2020-04-13,LAC -- LACTATO,0.9,mmol/L,(0.5 - 2.4),F_19_04_2021
1146936,621,I-96,2020-04-13,PCO2 -- GASOMETRIA ARTERIAL,49.0,mmHg,(35.0 - 45.0),F_19_04_2021
1146937,621,I-96,2020-04-13,PCO2 -- pCO2,49.0,mmHg,(35.0 - 45.0),F_19_04_2021


In [65]:
df_laboratorio_tratado[(df_laboratorio_tratado['ID_PACIENTE'] == 745) ]

,ID_PACIENTE,LAB_NUMERO,LAB_FECHA,LAB_ITEM,RESULTADO_VALOR,RESULTADO_UNIDAD,RESULTADO_VALORES_REFERENCIALES,FUENTE
90825,745,I-2,2020-03-25,ADW -- Coeficiente de anisocitosis,14.3,%,(11.5 - 14.5),F_24_04_2020
90826,745,I-2,2020-03-25,GLU -- GLUCOSA,87.6,mg/dL,(70.0 - 105.0),F_24_04_2020
90827,745,I-2,2020-03-25,NA -- SODIO,136.4,mmol/L,(135.0 - 145.0),F_24_04_2020
90828,745,I-2,2020-03-25,NEU -- Neutrófilos,4.08,x10e3/µL,(1.5 - 7.5),F_24_04_2020
90829,745,I-2,2020-03-25,NEU% -- Neutrófilos %,71.2,%,(40.0 - 75.0),F_24_04_2020
...,...,...,...,...,...,...,...,...
1175463,745,I-9,2020-03-10,PTORS -- Proteinas,75 mg/dl,NaN,NaN,F_19_04_2021
1175464,745,I-9,2020-03-10,SED1 --,Leucocitos: de 1 a 5/c,NaN,NaN,F_19_04_2021
1175465,745,I-9,2020-03-10,SED2 --,Hemat°es: de 1 a 5/c,NaN,NaN,F_19_04_2021
1175466,745,I-9,2020-03-10,SED3 --,CÇlulas epiteliales escamosas,NaN,NaN,F_19_04_2021


from matplotlib import pyplot as plt
import seaborn as sns
_df_0.groupby('FUENTE').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['LAB_FECHA']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'LAB_FECHA'}, axis=1)
              .sort_values('LAB_FECHA', ascending=True))
  xs = counted['LAB_FECHA']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_1.sort_values('LAB_FECHA', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('FUENTE')):
  _plot_series(series, series_name, i)
  fig.legend(title='FUENTE', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('LAB_FECHA')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['ID_PACIENTE']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'ID_PACIENTE'}, axis=1)
              .sort_values('ID_PACIENTE', ascending=True))
  xs = counted['ID_PACIENTE']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_2.sort_values('ID_PACIENTE', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('FUENTE')):
  _plot_series(series, series_name, i)
  fig.legend(title='FUENTE', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('ID_PACIENTE')
_ = plt.ylabel('count()')

In [ ]:
#LAB_ITEM

df_laboratorio_tratado['LAB_ITEM'] = df_laboratorio_tratado['LAB_ITEM'].apply(limpiar_texto)
df_laboratorio_tratado.value_counts("LAB_ITEM")

,count
LAB_ITEM,
K -- POTASIO,39196
NA -- SODIO,39118
CREA -- CREATININA,38144
U -- UREA,37214
PCR -- PROTEINA C REACTIVA,36068
...,...
LEGIONELLA -- CULTIVO DE LEGIONELLA,1
METAFASES -- CARIOTIPO EN MEDULA OSEA,1
BASEMOL -- PCR RAPIDA VIRUS CORONAVIRUS 2019NCOV,1


In [78]:
df_laboratorio_tratado['RESULTADO_VALOR'] = df_laboratorio_tratado['RESULTADO_VALOR'].apply(limpiar_texto)
df_laboratorio_tratado['RESULTADO_VALOR'] = df_laboratorio_tratado['RESULTADO_VALOR'].str.strip().replace('TP; INR', 'TP;INR')
df_laboratorio_tratado.value_counts("RESULTADO_VALOR")

,count
RESULTADO_VALOR,
0.0,29873
0.01,15497
0.2,11836
0.1,11609
0.02,10165
...,...
"POBLACIÓN ANALIZADA: LINFOCITARIA (30.2%). SE IDENTIFICAN LAS SIGUIENTES SUBPOBLACIONES: LINFOCITOS T CD3+ (79.09%), 1060 CELS/UL - LINFOCITOS T CD4+ (51.78%), 549 CELS/UL - LINFOCITOS T CD8+ (46.73%), 495 CELS/UL LINFOCITOS B CD19+ CD20+ (13.38%), 179 CELS/UL LINFOCITOS NK T CD3+ CD56+ (2.84%), 38 CELS/UL CÉLULAS NK CD3- CD56+ (3.08%), 41 CELS/UL",1
POBLACIÓN ANALIZADA: LINFOCITARIA (30.66%). SE IDENTIFICAN LAS SIGUIENTES SUBPOBLACIONES: LINFOCITOS T CD3+ (75.9%),1
"POBLACIÓN ANALIZADA: LINFOCITARIA (30.66%). SE IDENTIFICAN LAS SIGUIENTES SUBPOBLACIONES: LINFOCITOS T CD3+ (75.9%), 486 CELS/UL - LINFOCITOS T CD4+ (81.7%), 397 CELS/UL - LINFOCITOS T CD8+ (14.7%), 71 CELS/UL LINFOCITOS B CD19+ CD20+ (8.9%), 57 CELS/UL LINFOCITOS NK T CD3+ CD56+ (5.30%), 34 CELS/UL CÉLULAS NK CD3- CD56+ (6.1%), 39 CELS/UL",1


In [79]:
df_laboratorio_tratado['RESULTADO_UNIDAD'] = df_laboratorio_tratado['RESULTADO_UNIDAD'].apply(limpiar_texto)
df_laboratorio_tratado.value_counts("RESULTADO_UNIDAD")

,count
RESULTADO_UNIDAD,
%,325547
MMOL/L,242263
MG/DL,209773
U/L,161462
X10E3/ÁL,151265
...,...
75.6,1
25 % PROMIELOCITOS; 19 % MIELOCITOS; 11 % METAMIELOCITOS; 6,1
513.0,1


In [80]:
df_laboratorio_tratado['RESULTADO_VALORES_REFERENCIALES'] = df_laboratorio_tratado['RESULTADO_VALORES_REFERENCIALES'].apply(limpiar_texto)
df_laboratorio_tratado.value_counts("RESULTADO_VALORES_REFERENCIALES")

,count
RESULTADO_VALORES_REFERENCIALES,
(-2.0 - 3.0),58884
(150.0 - 450.0),52156
(3.5 - 5.1),46176
<41,43172
(135.0 - 145.0),39058
...,...
1% METAMIELO,1
100 CELS/UL - LINFOCITOS T CD8+ (44.67%),1
1058CELS/UL - LINFOCITOS T CD8+ (19.16%),1


In [82]:
df_laboratorio_tratado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1767082 entries, 0 to 1767081
Data columns (total 8 columns):
 #   Column                           Dtype         
---  ------                           -----         
 0   ID_PACIENTE                      int64         
 1   LAB_NUMERO                       object        
 2   LAB_FECHA                        datetime64[ns]
 3   LAB_ITEM                         object        
 4   RESULTADO_VALOR                  object        
 5   RESULTADO_UNIDAD                 object        
 6   RESULTADO_VALORES_REFERENCIALES  object        
 7   FUENTE                           object        
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 107.9+ MB


In [84]:
df_laboratorio_tratado.to_csv(ruta + '/04_laboratorio_limpio.csv', sep=";", encoding='utf-8-sig', index=False, date_format="%Y-%m-%d %H:%M:%S")

## 5. CIE10 Urgencias

In [ ]:
url = "05_cie10_urgencia.csv";
df_cie10_urgencia_original =  pd.read_csv(url,delimiter=DetectaSeparador(url),encoding=DetectaEncoding(url))
df_cie10_urgencia_tratado = df_cie10_urgencia_original.copy()
df_cie10_urgencia_original

,ID_PACIENTE,DIA_PPAL,DIA_02,DIA_03,DIA_04,DIA_05,DIA_06,DIA_07,DIA_08,DIA_09,DIA_10,DIA_11,DIA_12,PROC_01,PROC_02,PROC_03,PROC_04,PROC_05,FUENTE
0,47,J12.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F_24_04_2020
1,48,J98.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F_24_04_2020
2,49,I50.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F_24_04_2020
3,50,J98.4,N39.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F_24_04_2020
4,51,J18.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F_24_04_2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8633,2758,E86.0,E87.1,J12.89,J84.9,R68.89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BW03ZZZ,0BJ0XZZ,NaN,NaN,NaN,F_19_04_2021
8634,2759,J96.90,R68.89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BW03ZZZ,NaN,NaN,NaN,NaN,F_19_04_2021
8635,2760,R06.00,J12.89,B97.29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BW03ZZZ,NaN,NaN,NaN,NaN,F_19_04_2021
8636,2761,R68.89,J96.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BW03ZZZ,NaN,NaN,NaN,NaN,F_19_04_2021


In [ ]:
EDA(df_cie10_urgencia_tratado, "CIE10 Urgencia")

In [ ]:
df_cie10_urgencia_tratado.to_csv(ruta + '/05_cie10_urgencia_limpio.csv', sep=";", encoding='utf-8-sig', index=False)

## 6. CIE10 Ingresos

In [ ]:
url = "06_cie10_ingreso.csv";
df_cie10_ingreso_original =  pd.read_csv(url,delimiter=DetectaSeparador(url),encoding=DetectaEncoding(url))
df_cie10_ingreso_tratado = df_cie10_ingreso_original.copy()
df_cie10_ingreso_original

<ipython-input-42-98c3c92890b8>:2: DtypeWarning: Columns (23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cie10_ingreso_original =  pd.read_csv(url,delimiter=DetectaSeparador(url),encoding=DetectaEncoding(url))


,ID_PACIENTE,DIA_PPAL,DIA_02,DIA_03,DIA_04,DIA_05,DIA_06,DIA_08,DIA_07,DIA_09,...,PROC_12,PROC_13,PROC_14,PROC_15,PROC_16,PROC_17,PROC_18,PROC_19,PROC_20,FUENTE
0,2293,J18.9,Z20.828,R68.89,R09.02,D80.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F_24_04_2020
1,2287,J12.89,B97.29,M19.90,F32.9,G25.81,M51.86,Z96.1,Z98.49,Z60.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F_24_04_2020
2,2269,J18.9,R68.89,J96.90,I10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F_24_04_2020
3,2265,J12.89,B97.29,J96.00,J84.89,Z99.89,Z95.0,NaN,Z79.52,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F_24_04_2020
4,2264,J47.0,J12.89,B97.29,J47.1,I10,E11.9,Z87.891,K44.9,Z98.52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F_24_04_2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8720,2904,R05,Z20.828,R68.89,K29.50,Z98.52,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F_19_04_2021
8721,2912,J98.8,R68.89,Z20.828,R79.89,L50.3,Z86.19,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F_19_04_2021
8722,2916,G44.89,R68.89,Z20.828,R78.81,B95.7,M93.88,M47.812,I35.0,F33.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F_19_04_2021
8723,2956,J18.9,Z20.828,I48.91,J90,I11.0,I50.9,H81.399,E11.9,H91.90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F_19_04_2021


In [ ]:
EDA(df_cie10_ingreso_tratado, "CIE10 Ingreso")

In [ ]:
df_cie10_ingreso_tratado.to_csv(ruta + '/06_cie10_ingreso_limpio.csv', sep=";", encoding='utf-8-sig', index=False)

## 7. CIE10 Códigos Diagnosticos

In [ ]:
url = "07_cie10_cod_diag.csv";
df_cie10_diagnosticos_original =  pd.read_csv(url,delimiter=DetectaSeparador(url),encoding=DetectaEncoding(url))
df_cie10_diagnosticos_tratado = df_cie10_diagnosticos_original.copy()
df_cie10_diagnosticos_original

,ID,CODIGO,NIVEL,DESCRIPCION_CORTA,DESCRIPCION_LARGA
0,1,A00,0,Cholera,Cholera
1,2,A000,1,"Cholera due to Vibrio cholerae 01, biovar chol...","Cholera due to Vibrio cholerae 01, biovar chol..."
2,3,A001,1,"Cholera due to Vibrio cholerae 01, biovar eltor","Cholera due to Vibrio cholerae 01, biovar eltor"
3,4,A009,1,"Cholera, unspecified","Cholera, unspecified"
4,5,A01,0,Typhoid and paratyphoid fevers,Typhoid and paratyphoid fevers
...,...,...,...,...,...
95353,95354,Z9981,1,Dependence on supplemental oxygen,Dependence on supplemental oxygen
95354,95355,Z9989,1,Dependence on other enabling machines and devices,Dependence on other enabling machines and devices
95355,95356,U07,0,Emergency use of U07,Emergency use of U07
95356,95357,U070,1,Vaping-related disorder,Vaping-related disorder


In [ ]:
EDA(df_cie10_diagnosticos_tratado, "CIE10 Códigos Diagnosticos")

In [ ]:
df_cie10_diagnosticos_tratado['DESCRIPCION_CORTA'] = df_cie10_diagnosticos_tratado['DESCRIPCION_CORTA'].apply(limpiar_texto)
df_cie10_diagnosticos_tratado.value_counts("DESCRIPCION_CORTA")

,count
DESCRIPCION_CORTA,
"INJ BLOOD VESSELS AT ABDOMEN, LOW BACK AND PELVIS LEVEL",3
"OTHER FAMILY MEMBER, PERPETRATOR OF MALTREATMENT AND NEGLECT",3
PERSONAL HISTORY OF OTHER SPECIFIED CONDITIONS,3
OTHER GENERAL SYMPTOMS AND SIGNS,3
OTHER FALL FROM ONE LEVEL TO ANOTHER,3
...,...
HEMORRHOIDS IN THE PUERPERIUM,1
"HEMORRHOIDS IN PREGNANCY, UNSPECIFIED TRIMESTER",1
"HEMORRHOIDS IN PREGNANCY, THIRD TRIMESTER",1


In [ ]:
df_cie10_diagnosticos_tratado['DESCRIPCION_LARGA'] = df_cie10_diagnosticos_tratado['DESCRIPCION_LARGA'].apply(limpiar_texto)
df_cie10_diagnosticos_tratado.value_counts("DESCRIPCION_LARGA")

,count
DESCRIPCION_LARGA,
OTHER DIFFICULTIES WITH MICTURITION,3
OTHER GENERAL SYMPTOMS AND SIGNS,3
ASSAULT BY OTHER SPECIFIED MEANS,3
OTHER LONG TERM (CURRENT) DRUG THERAPY,3
"OTHER FAMILY MEMBER, PERPETRATOR OF MALTREATMENT AND NEGLECT",3
...,...
HEREDITARY RETINAL DYSTROPHY,1
HEREDITARY PERSISTENCE OF FETAL HEMOGLOBIN [HPFH],1
HEREDITARY OPTIC ATROPHY,1


In [ ]:
df_cie10_diagnosticos_tratado.to_csv(ruta + '/07_cie10_codigos_diagnosticos_limpio.csv', sep=";", encoding='utf-8-sig', index=False)